# Fase 1 – Creación de Archivos
Crear carpetas data, database, reports, ci, git y scripts y estructura base del proyecto usando os y pathlib en tiempo de ejecución


In [1]:
from pathlib import Path

# Crear estructura base
base_path = Path("SaludDigitalB_E1")

folders = [
    "data", "database", "reports", "ci", "git", "scripts"
]

for folder in folders:
    (base_path / folder).mkdir(parents=True, exist_ok=True)

print("Estructura creada correctamente ✅")


Estructura creada correctamente ✅


# Fase 2 – Definición de Esquema
Generar un archivo base CSV (base.csv) con la estructura indicada.


In [2]:
import pandas as pd

columnas = [
    "id_paciente", "nombre", "edad", "sexo",
    "distrito", "servicio", "fecha_atencion", "monto"
]

df_base = pd.DataFrame(columns=columnas)
df_base.to_csv("SaludDigitalB_E1/data/base.csv", index=False)

print("Archivo base.csv creado correctamente ✅")


Archivo base.csv creado correctamente ✅


# Fase 3 – Generación de Datos Aleatorios
Generar un archivo CSV (pacientes.csv) con 3000 registros simulados, que incluya campos nulos, NA, entre otros


In [7]:
# ...existing code...
import pandas as pd
import numpy as np
import random
from faker import Faker

fake = Faker('es_ES')
servicios = ["Odontología", "Pediatría", "Medicina General", "Cardiología", "Dermatología"]
distritos = ["Miraflores", "Surco", "San Isidro", "La Molina", "Barranco"]

data = []
for i in range(3000):
    data.append([
        f"P{i+1:04d}",
        fake.name(),
        np.random.randint(1, 90),
        random.choice(["M", "F"]),
        random.choice(distritos),
        random.choice(servicios),
        fake.date_between(start_date="-2y", end_date="today"),
        round(np.random.uniform(50, 500), 2)
    ])

df = pd.DataFrame(data, columns=[
    "id_paciente", "nombre", "edad", "sexo",
    "distrito", "servicio", "fecha_atencion", "monto"
])

# Introducir valores nulos aleatorios
for col in ["sexo", "distrito"]:
    df.loc[df.sample(frac=0.02, random_state=1).index, col] = np.nan

df.to_csv("SaludDigitalB_E1/data/pacientes.csv", index=False)
print("Archivo pacientes.csv generado correctamente ✅")


Archivo pacientes.csv generado correctamente ✅


# Fase 4 – ETL (Limpieza de Datos)
•	Eliminar duplicados
•	Rellenar valores nulos en campos de transporte o destino
•	Formatear fechas
•	Validar tipos de datos
Guardar como pacientes_clean.csv.


In [8]:
# Extracción
df = pd.read_csv("SaludDigitalB_E1/data/pacientes.csv")

# Transformación
df.drop_duplicates(inplace=True)
df["sexo"].fillna("No especifica", inplace=True)
df["distrito"].fillna("Sin distrito", inplace=True)
df["fecha_atencion"] = pd.to_datetime(df["fecha_atencion"], errors="coerce")
df["edad"] = df["edad"].astype(int)

# Carga
df.to_csv("SaludDigitalB_E1/database/pacientes_clean.csv", index=False)
print("Datos limpios y guardados correctamente ✅")


Datos limpios y guardados correctamente ✅


C:\Users\Hayler Perez\AppData\Local\Temp\ipykernel_24276\1760083827.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["sexo"].fillna("No especifica", inplace=True)
C:\Users\Hayler Perez\AppData\Local\Temp\ipykernel_24276\1760083827.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

# Fase 5 – Carga en MongoDB
Crear base de datos SaludDigital_2025 y colección pacientes
Insertar los registros limpios desde CSV.


In [9]:
from pymongo import MongoClient
import pandas as pd

# Conexión
client = MongoClient("mongodb://localhost:27017/")
db = client["SaludDigital_2025"]
collection = db["pacientes"]

# Cargar datos limpios
df = pd.read_csv("SaludDigitalB_E1/database/pacientes_clean.csv")
collection.insert_many(df.to_dict("records"))

print("Datos cargados en MongoDB correctamente ✅")


Datos cargados en MongoDB correctamente ✅


# Fase 6 – Visualización de Datos
Usar matplotlib y pandas para mostrar:
1.	Gráfico de barras: número de pacientes atendidos por servicio.
2.	Gráfico circular: distribución por sexo.
3.	Histograma: distribución de edades.


In [10]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("SaludDigitalB_E1/database/pacientes_clean.csv")

# Gráfico de barras
df["servicio"].value_counts().plot(kind="bar")
plt.title("Pacientes atendidos por servicio")
plt.savefig("SaludDigitalB_E1/reports/pacientes_atendidos_por_servicio.png")
plt.close()

# Gráfico circular
df["sexo"].value_counts().plot(kind="pie", autopct='%1.1f%%')
plt.title("Distribución por sexo")
plt.savefig("SaludDigitalB_E1/reports/distribucion_por_sexo.png")
plt.close()

# Histograma
df["edad"].plot(kind="hist", bins=10)
plt.title("Distribución de edades")
plt.savefig("SaludDigitalB_E1/reports/distribucion_de_edades.png")
plt.close()

# Promedio de montos por distrito
promedio = df.groupby("distrito")["monto"].mean().reset_index()
promedio.to_csv("SaludDigitalB_E1/reports/promedio_de_montos_por_distrito.csv", index=False)
print("Reportes generados correctamente ✅")


Reportes generados correctamente ✅
